In [1]:
import numpy as np
from gurobipy import * 
import pandas as pd
pd.set_option('display.max_rows',None)
import matplotlib.pyplot as plt
import random
from random import seed
from random import choices



In [6]:
def cell_manage(p_s,p_end,total_t,xp,xw,ss,w_s,w_lc,w_i):

    MODEL = gurobipy.Model("Lane_Manage")
    MODEL.Params.timeLimit = 100
    MODEL.Params.MIPGap = 0.1
    MODEL.params.NonConvex = 2
    MODEL.params.ConcurrentMIP = 4

    ##################################################

    #index

    T_end = 8
    J_end = 6
    I_end = 4
    I = range(I_end)
    J = range(J_end+2)
    T = range(T_end)
    D = [-1,1]#through left


    n = {}
    y = {}
    s = {}
    r = {}
    k = {}
    q = {}
    z = {}
    h = {}
    r_list = {}
    s_list = {}
    aux_r = {}
    aux_n = {}
    v_t = {}
    v_t_1 = {}


    k_max = 324.3 #veh/mile
    q_max = 3600 #veh/h
    v = 40 #mile
    w = 15.365 
    delta_t = 5/3600 #per h
    delta_l =  v*delta_t # mile
    a_max =24112.6




    for i in I:
        for j in J:
            for d in D:
                for t in T:
                    n[i,j,d,t] = MODEL.addVar(name='n_%s_%s_%s_%s' % (i,j,d,t))
                    k[i,j,d,t] = MODEL.addVar(name='k_%s_%s_%s_%s' % (i,j,d,t))
                    z[i,j,d,t] = MODEL.addVar(vtype=GRB.BINARY, name='z_%s_%s_%s_%s' % (i,j,d,t))
                    r_list[i,j,d,t] = MODEL.addVar(name='r_list_%s_%s_%s_%s' % (i,j,d,t))
                    s_list[i,j,d,t] = MODEL.addVar(name='s_list_%s_%s_%s_%s' % (i,j,d,t))
                    aux_r[i,j,d,t] = MODEL.addVar(name='aux_r_%s_%s_%s_%s' % (i,j,d,t))
                    aux_n[i,j,d,t] = MODEL.addVar(name='aux_n_%s_%s_%s_%s' % (i,j,d,t))
                    v_t[i,j,t] = MODEL.addVar(name='v_t_%s_%s_%s' % (i,j,t))
                    v_t_1[i,j,t] = MODEL.addVar(name='v_t_1_%s_%s_%s' % (i,j,t))



                    
                    for i_d in I:
                        j_d = j+1
                        if (i_d - i <= 1) and (i_d - i >= -1):
                            q[i,j,i_d,j_d,d,t] = MODEL.addVar(name='q_%s_%s_%s_%s_%s_%s' % (i,j,i_d,j_d,d,t))
                            y[i,j,i_d,j_d,d,t] = MODEL.addVar(name='y_%s_%s_%s_%s_%s_%s' % (i,j,i_d,j_d,d,t))
                            r[i,j,i_d,j_d,d,t] = MODEL.addVar(name='r_%s_%s_%s_%s_%s_%s' % (i,j,i_d,j_d,d,t))
                            s[i,j,i_d,j_d,d,t] = MODEL.addVar(name='s_%s_%s_%s_%s_%s_%s' % (i,j,i_d,j_d,d,t))
                            h[i,j,i_d,j_d,d,t] = MODEL.addVar(vtype=GRB.BINARY,name='h_%s_%s_%s_%s_%s_%s' % (i,j,i_d,j_d,d,t))



    MODEL.update()


    n_last_sum = quicksum((T_end-t)*n[i,J_end-1,d,t] for i in I for t in T for d in D)
    cost_lane_change = quicksum(y[i,j,i-1,j+1,d,t] for i in [1,2,3] for t in T for d in D for j in J)+quicksum(y[i,j,i+1,j+1,d,t] for i in [0,1,2] for t in T for d in D for j in J)
    cost_j = quicksum((- j)*n[i,j,d,t] for i in I for t in T for d in D for j in range(2,J_end))
    y_1 = quicksum(y[i,j,i,j+1,d,t] for i in I for t in T for d in D for j in [1]) + quicksum(y[i,j,i+1,j+1,d,t] for i in [0,1,2] for t in T for d in D for j in [1]) + quicksum(y[i,j,i-1,j+1,d,t] for i in [3,1,2] for t in T for d in D for j in [1])
    obj = w_s*n_last_sum - w_lc*cost_lane_change+w_i*y_1




    MODEL.setObjective(obj, GRB.MAXIMIZE)
    for i in I:
        for t in T:
            for d in D:
                MODEL.addConstr(n[i,J_end+1,d,t]==0)
                
    #input
    for i in I:
        for d in D:
            for j in J:
                MODEL.addConstr(n[i,j,d,0]==0)




    b = MODEL.addVar(vtype=GRB.BINARY, name="b")

    # Constants
    eps = 0.00001
    M = delta_t*q_max + eps 
    for t in range(T_end):
        i=0
        y1[sepa+t] = round(y1[sepa+t])

        MODEL.addConstr(y1[sepa+t] >= r[i,0,i,1,dl1[sepa+t],t] + eps - M * (1 - b), name="bigM_constr1")
        MODEL.addConstr(y1[sepa+t] <= r[i,0,i,1,dl1[sepa+t],t] + M * b, name="bigM_constr2")

        # Add indicator constraints
        MODEL.addConstr((b == 1) >> (y[i,0,i,1,dl1[sepa+t],t] == r[i,0,i,1,dl1[sepa+t],t]), name="indicator_constr1")
        MODEL.addConstr((b == 0) >> (y[i,0,i,1,dl1[sepa+t],t] == y1[sepa+t]), name="indicator_constr2")
        MODEL.addConstr((y[i,0,i,1,-dl1[sepa+t],t] == 0))

        i=1
        y2[sepa+t] = round(y2[sepa+t])

        MODEL.addConstr(y2[sepa+t] >= r[i,0,i,1,dl2[sepa+t],t] + eps - M * (1 - b), name="bigM_constr1")
        MODEL.addConstr(y2[sepa+t] <= r[i,0,i,1,dl2[sepa+t],t] + M * b, name="bigM_constr2")

        # Add indicator constraints
        MODEL.addConstr((b == 1) >> (y[i,0,i,1,dl2[sepa+t],t] == r[i,0,i,1,dl1[sepa+t],t]), name="indicator_constr1")
        MODEL.addConstr((b == 0) >> (y[i,0,i,1,dl2[sepa+t],t] == y2[sepa+t]), name="indicator_constr2")
        MODEL.addConstr((y[i,0,i,1,-dl2[sepa+t],t] == 0))

        i=2
        y3[sepa+t] = round(y3[sepa+t])

        MODEL.addConstr(y3[sepa+t] >= r[i,0,i,1,dl3[sepa+t],t] + eps - M * (1 - b), name="bigM_constr1")
        MODEL.addConstr(y3[sepa+t] <= r[i,0,i,1,dl3[sepa+t],t] + M * b, name="bigM_constr2")

        # Add indicator constraints
        MODEL.addConstr((b == 1) >> (y[i,0,i,1,dl3[sepa+t],t] == r[i,0,i,1,dl1[sepa+t],t]), name="indicator_constr1")
        MODEL.addConstr((b == 0) >> (y[i,0,i,1,dl3[sepa+t],t] == y3[sepa+t]), name="indicator_constr2")
        MODEL.addConstr((y[i,0,i,1,-dl3[sepa+t],t] == 0))

        i=3
        y4[sepa+t] = round(y4[sepa+t])

        MODEL.addConstr(y4[sepa+t] >= r[i,0,i,1,dl4[sepa+t],t] + eps - M * (1 - b), name="bigM_constr1")
        MODEL.addConstr(y4[sepa+t] <= r[i,0,i,1,dl4[sepa+t],t] + M * b, name="bigM_constr2")

        # Add indicator constraints
        MODEL.addConstr((b == 1) >> (y[i,0,i,1,dl4[sepa+t],t] == r[i,0,i,1,dl1[sepa+t],t]), name="indicator_constr1")
        MODEL.addConstr((b == 0) >> (y[i,0,i,1,dl4[sepa+t],t] == y4[sepa+t]), name="indicator_constr2")
        MODEL.addConstr((y[i,0,i,1,-dl4[sepa+t],t] == 0))



    for d in D:
        for i in I:
            for t in T:
                MODEL.addConstr(n[i,0,d,t]==0)



    for i in I:
        for t in T:
            sg = ss[sepa][t]
            if sg == 1:
                MODEL.addConstr(y[i,J_end-2,i,J_end-1,1,t]<=0)
            elif sg == 2:
                MODEL.addConstr(y[i,J_end-2,i,J_end-1,-1,t]<=0)
            elif sg == 0:
                MODEL.addConstr(y[i,J_end-2,i,J_end-1,1,t]<=0)
                MODEL.addConstr(y[i,J_end-2,i,J_end-1,-1,t]<=0)


            

    #cannot cross lane change
    for j in J:
        for t in T:
            for d in D:
                i=0
                MODEL.addConstr(y[i,j,i+1,j+1,d,t] <= h[i,j,i+1,j+1,d,t]*y[i,j,i+1,j+1,d,t])
                MODEL.addConstr(y[i,j,i,j+1,d,t] <= h[i,j,i,j+1,d,t]*y[i,j,i,j+1,d,t])
                i=I_end-1
                MODEL.addConstr(y[i,j,i-1,j+1,d,t] <= h[i,j,i-1,j+1,d,t]*y[i,j,i-1,j+1,d,t])
                MODEL.addConstr(y[i,j,i,j+1,d,t] <= h[i,j,i,j+1,d,t]*y[i,j,i,j+1,d,t])
                for i in range(1,I_end-1):
                    for i_d in [i+1,i-1,i]:
                        MODEL.addConstr(y[i,j,i_d,j+1,d,t] <= h[i,j,i_d,j+1,d,t]*y[i,j,i_d,j+1,d,t])


    for j in range(0,J_end+1):
        for t in T:
            i = 0
            MODEL.addConstr(quicksum(h[i,j,i+1,j+1,d,t]+h[i+1,j,i,j+1,d,t] for d in D) <= 1)
            i = I_end-1
            MODEL.addConstr(quicksum(h[i,j,i-1,j+1,d,t]+h[i-1,j,i,j+1,d,t] for d in D) <= 1)
            for i in range(1,I_end-1):
                MODEL.addConstr(quicksum(h[i,j,i+1,j+1,d,t]+h[i+1,j,i,j+1,d,t] for d in D) <= 1)
                MODEL.addConstr(quicksum(h[i,j,i-1,j+1,d,t]+h[i-1,j,i,j+1,d,t] for d in D) <= 1)

    for j in range(0,J_end+1):
        for t in T:
            for d in D:
                i = 0
                MODEL.addConstr(h[i,j,i,j+1,d,t]+h[i+1,j,i,j+1,-d,t] <= 1)
                i = I_end-1
                MODEL.addConstr(h[i,j,i,j+1,d,t]+h[i-1,j,i,j+1,-d,t] <= 1)
                for i in range(1,I_end-1):
                    MODEL.addConstr(h[i,j,i,j+1,d,t]+h[i+1,j,i,j+1,-d,t]  <= 1)
                    MODEL.addConstr(h[i,j,i,j+1,d,t]+h[i-1,j,i,j+1,-d,t]  <= 1)
                    MODEL.addConstr(h[i-1,j,i,j+1,d,t]+h[i+1,j,i,j+1,-d,t]  <= 1)
                

    for i in I:
        for j in J:
            for t in T:
                for d in D:
                    MODEL.addConstr(n[i,j,d,t] <= z[i,j,d,t]*n[i,j,d,t])

    for i in I:
        for j in range(2,J_end+2):
            for t in T:
                MODEL.addConstr(quicksum(z[i,j,d,t] for d in D) <= 1)

    for i in I:
        for j in [0,1]:
            for t in T:
                MODEL.addConstr(quicksum(z[i,j,d,t] for d in D) <= 2)

    for t in T:
        for j in [0, J_end-1,J_end-2]:
            for d in D:
                for i in range(1,I_end-1):
                    MODEL.addConstr(y[i,j,i+1,j+1,d,t]==0)
                    MODEL.addConstr(y[i,j,i-1,j+1,d,t]==0)
                    MODEL.addConstr(y[i,j,i,j+1,d,t]>=0)
                i=0
                MODEL.addConstr(y[i,j,i+1,j+1,d,t]==0)
                MODEL.addConstr(y[i,j,i,j+1,d,t]>=0)
                i=I_end-1
                MODEL.addConstr(y[i,j,i-1,j+1,d,t]==0)
                MODEL.addConstr(y[i,j,i,j+1,d,t]>=0)



    #same v in a cell
    for j in [0,1]:
        for t in T:
            i = 0
            MODEL.addConstr((y[i,j,i,j+1,1,t]+y[i,j,i+1,j+1,1,t])*n[i,j,-1,t] == (y[i,j,i,j+1,-1,t]+y[i,j,i+1,j+1,-1,t])*n[i,j,1,t])
            i = I_end-1
            MODEL.addConstr((y[i,j,i-1,j+1,1,t]+y[i,j,i,j+1,1,t])*n[i,j,-1,t] == (y[i,j,i-1,j+1,-1,t]+y[i,j,i,j+1,-1,t])*n[i,j,1,t])
            for i in range(1,I_end-1):
                MODEL.addConstr((y[i,j,i-1,j+1,1,t]+y[i,j,i,j+1,1,t]+y[i,j,i+1,j+1,1,t])*n[i,j,-1,t] == (y[i,j,i-1,j+1,-1,t]+y[i,j,i,j+1,-1,t]+y[i,j,i+1,j+1,-1,t])*n[i,j,1,t])
            
                



    #dynamic
    #############################
    #set receiving flow R, sending flow S
    for j in range(1,J_end+1):
        for t in T:
            for i in I:
                MODEL.addConstr(aux_n[i,j,d,t] == n[i,j,1,t]+n[i,j,-1,t])
                MODEL.addGenConstrMin(s_list[i,j,d,t],[aux_n[i,j,d,t],q_max*delta_t])

                i = 0   
                MODEL.addConstr(quicksum(s[i,j,i,j+1,d,t] + s[i,j,i+1,j+1,d,t] for d in D)== s_list[i,j,d,t])

                i = 3    
                MODEL.addConstr(quicksum(s[i,j,i,j+1,d,t] + s[i,j,i-1,j+1,d,t] for d in D)== s_list[i,j,d,t])

                for i in [1,2]:   
                    MODEL.addConstr(quicksum(s[i,j,i,j+1,d,t] + s[i,j,i-1,j+1,d,t] + s[i,j,i+1,j+1,d,t] for d in D) == s_list[i,j,d,t])


    for j in range(J_end+1):
        for t in T:
            for i in I:
                MODEL.addConstr(aux_r[i,j,d,t] == w/v*(k_max*delta_l-(n[i,j,1,t]+n[i,j,-1,t])))
                MODEL.addGenConstrMin(r_list[i,j,d,t],[aux_r[i,j,d,t],q_max*delta_t])

                i = 0
                MODEL.addConstr(quicksum(r[i,j,i,j+1,d,t] + r[i+1,j,i,j+1,d,t] for d in D) == r_list[i,j+1,d,t])

                i = 3
                MODEL.addConstr(quicksum(r[i,j,i,j+1,d,t] + r[i-1,j,i,j+1,d,t] for d in D) ==r_list[i,j+1,d,t])

                
                for i in [1,2]: 
                    MODEL.addConstr(quicksum(r[i,j,i,j+1,d,t] + r[i-1,j,i,j+1,d,t] + r[i+1,j,i,j+1,d,t] for d in D)==r_list[i,j+1,d,t])

    for j in range(J_end+1):
        for t in T:
            for d in D:
                for i in I:
                    for i_d in I:
                        if (i_d - i <= 1) and (i_d - i >= -1):
                            MODEL.addGenConstrMin(y[i,j,i_d,j+1,d,t],[s[i,j,i_d,j+1,d,t],r[i,j,i_d,j+1,d,t]])




    for j in range(1,J_end+1):
        for t in T:
            for d in D:
                i=0
                MODEL.addConstr(n[i,j,d,t] >= y[i,j,i,j+1,d,t]+y[i,j,i+1,j+1,d,t])
                i=I_end-1
                MODEL.addConstr(n[i,j,d,t] >= y[i,j,i,j+1,d,t]+y[i,j,i-1,j+1,d,t])
                for i in range(1,I_end-1):
                    MODEL.addConstr(n[i,j,d,t] >= y[i,j,i,j+1,d,t]+y[i,j,i-1,j+1,d,t]+y[i,j,i+1,j+1,d,t]) 

    for j in range(1,J_end+1):
        for t in range(1,T_end):
            for d in D:
                i=0
                MODEL.addConstr(n[i,j,d,t] == n[i,j,d,t-1]+(y[i,j-1,i,j,d,t-1]+y[i+1,j-1,i,j,d,t-1])-(y[i,j,i,j+1,d,t-1]+y[i,j,i+1,j+1,d,t-1]))
                i=I_end-1
                MODEL.addConstr(n[i,j,d,t] == n[i,j,d,t-1]+(y[i,j-1,i,j,d,t-1]+y[i-1,j-1,i,j,d,t-1])-(y[i,j,i,j+1,d,t-1]+y[i,j,i-1,j+1,d,t-1]))

                for i in range(1,I_end-1):
                    MODEL.addConstr(n[i,j,d,t] == n[i,j,d,t-1]+(y[i-1,j-1,i,j,d,t-1]+y[i,j-1,i,j,d,t-1]+y[i+1,j-1,i,j,d,t-1])-(y[i,j,i-1,j+1,d,t-1]+y[i,j,i,j+1,d,t-1]+y[i,j,i+1,j+1,d,t-1]))



    for i in I:
        for j in J:
            for t in T:
                for d in D:
                    MODEL.addConstr(n[i,j,d,t] == k[i,j,d,t]*delta_l)
                    for i_d in I:
                        if (i_d - i <= 1) and (i_d - i >= -1):
                            MODEL.addConstr(y[i,j,i_d,j+1,d,t] == q[i,j,i_d,j+1,d,t]*delta_t)
                            MODEL.addConstr(q[i,j,i_d,j+1,d,t] >=0)
                            MODEL.addConstr(quicksum(q[i,j,i_d,j+1,d,t] for d in D) <=q_max)
    for i in I:
        for j in J:
            for t in T:
                MODEL.addConstr(quicksum(k[i,j,d,t] for d in D) <= k_max)
                for d in D:
                    MODEL.addConstr(k[i,j,d,t] >= 0)




    for i in I:
        for d in D:
            for t in range(1,T_end):
                    MODEL.addConstr(n[i,J_end-1,d,t-1] == n[i,J_end,d,t])
                    MODEL.addConstr(n[i,J_end-1,d,t-1] == y[i,J_end-1,i,J_end,d,t-1])

    for tt in T:
        MODEL.addConstr(quicksum(y[i,0,i,1,d,t]for t in range(tt) for i in I for d in D) +  quicksum(n[i,j,d,0] for i in I for j in range(J_end-1) for d in D) == quicksum(y[i,J_end-2,i,J_end-1,d,t]
        for t in range(tt) for i in I for d in D) + quicksum(n[i,j,d,tt] for i in I for j in range(J_end-1) for d in D))

    #################################
    #add a related constr, v=f/d, a=(v-v)/t
    for i in I:
        for j in range(1,J_end+1):
            for t in range(0,T_end-1):
                eligible_i_d = [i_d for i_d in I if (i_d - i <= 1) and (i_d - i >= -1)]
                if eligible_i_d:
                    numerator = quicksum(q[i,j,i_d,j+1,d,t] for i_d in eligible_i_d for d in D)
                    denominator = quicksum(k[i,j,d,t] for d in D)
                    MODEL.addConstr(numerator == v_t[i,j,t] * denominator)

                    numerator_1 = quicksum(q[i,j,i_d,j+1,d,t+1] for i_d in eligible_i_d for d in D)
                    denominator_1 = quicksum(k[i,j,d,t+1] for d in D)
                    MODEL.addConstr(numerator_1 == v_t_1[i,j,t+1] * denominator_1)

                    MODEL.addConstr(v_t_1[i,j,t+1] - v_t[i,j,t+1] <=  a_max*delta_t)
                    MODEL.addConstr(v_t_1[i,j,t+1] - v_t[i,j,t+1] >= - a_max*delta_t)



    ################################


    MODEL.optimize()

    n_result = pd.DataFrame(columns=('i','j','d','t','value'))
    y_result = pd.DataFrame(columns=('i','j','i_d','j_d','t','value'))
    k_result = pd.DataFrame(columns=('i','j','d','t','value'))
    q_result = pd.DataFrame(columns=('i','j','i_d','j_d','t','value'))


    for t in T:


        for j in J:
            for i in I:
                for d in D:
                    value_n = n[i,j,d,t].X
                    n_result = n_result.append(pd.DataFrame({'i':[i],'j':[j],'d':[d],'t':[t],'value':[value_n]}),ignore_index=True)
                    
                    value_k = k[i,j,d,t].X
                    k_result = k_result.append(pd.DataFrame({'i':[i],'j':[j],'d':[d],'t':[t],'value':[value_k]}),ignore_index=True)


                    for i_d in I:
                        if (i_d - i <= 1) and (i_d - i >= -1):
                            value_y = round(y[i,j,i_d,j+1,d,t].X)
                            y_result = y_result.append(pd.DataFrame({'i':[i],'j':[j],'i_d':[i_d],'j_d':[j+1],'d':[d],'t':[t],'value':[value_y]}),ignore_index=True)

                            value_q = q[i,j,i_d,j+1,d,t].X
                            q_result = q_result.append(pd.DataFrame({'i':[i],'j':[j],'i_d':[i_d],'j_d':[j+1],'d':[d],'t':[t],'value':[value_q]}),ignore_index=True)






    for sep_idx in range(len(sep)):
        sepa = sep[sep_idx]

        n_result = n_result.drop(n_result[n_result['t']>=sepa].index)
        t = sepa
        for d in D:
            for j in [0, J_end+1]:
                for i in I:
                    n_result = n_result.append(pd.DataFrame({'i':[i],'j':[j],'d':[d],'t':[t],'value':[0]}),ignore_index=True)
            
            for j in range(1,J_end+1):
            
                i=0
                n_0 = n_result['value'][(n_result['i']==i) & (n_result['t']==t-1) & (n_result['d']==d) & (n_result['j'] ==j)].tolist()[0]
                y_ij_1 = y_result['value'][(y_result['i']==i) & (y_result['j'] ==j-1) & (y_result['i_d']==i) & (y_result['j_d'] ==j) & (y_result['t']==t-1) & (y_result['d']==d)].tolist()[0]
                y_i11j_1 = y_result['value'][(y_result['i']==i+1) & (y_result['j'] ==j-1) & (y_result['i_d']==i) & (y_result['j_d'] ==j) & (y_result['t']==t-1) & (y_result['d']==d)].tolist()[0]
                y_ij11 = y_result['value'][(y_result['i']==i) & (y_result['j'] ==j) & (y_result['i_d']==i) & (y_result['j_d'] ==j+1) & (y_result['t']==t-1) & (y_result['d']==d)].tolist()[0]
                y_i11j11 = y_result['value'][(y_result['i']==i) & (y_result['j'] ==j) & (y_result['i_d']==i+1) & (y_result['j_d'] ==j+1) & (y_result['t']==t-1) & (y_result['d']==d)].tolist()[0]
                n_1 = n_0 + (y_ij_1 + y_i11j_1) - (y_ij11 + y_i11j11)

                n_result = n_result.append(pd.DataFrame({'i':[i],'j':[j],'d':[d],'t':[t],'value':[n_1]}),ignore_index=True)


                i=3
                n_0 = n_result['value'][(n_result['i']==i) & (n_result['t']==t-1) & (n_result['d']==d) & (n_result['j'] ==j)].tolist()[0]
                y_ij_1 = y_result['value'][(y_result['i']==i) & (y_result['j'] ==j-1) & (y_result['i_d']==i) & (y_result['j_d'] ==j) & (y_result['t']==t-1) & (y_result['d']==d)].tolist()[0]
                y_i_1j_1 = y_result['value'][(y_result['i']==i-1) & (y_result['j'] ==j-1) & (y_result['i_d']==i) & (y_result['j_d'] ==j) & (y_result['t']==t-1) & (y_result['d']==d)].tolist()[0]
                y_ij11 = y_result['value'][(y_result['i']==i) & (y_result['j'] ==j) & (y_result['i_d']==i) & (y_result['j_d'] ==j+1) & (y_result['t']==t-1) & (y_result['d']==d)].tolist()[0]
                y_i_1j11 = y_result['value'][(y_result['i']==i) & (y_result['j'] ==j) & (y_result['i_d']==i-1) & (y_result['j_d'] ==j+1) & (y_result['t']==t-1) & (y_result['d']==d)].tolist()[0]
                n_1 = n_0 + (y_ij_1 + y_i_1j_1) - (y_ij11 + y_i_1j11)

                n_result = n_result.append(pd.DataFrame({'i':[i],'j':[j],'d':[d],'t':[t],'value':[n_1]}),ignore_index=True)

                for i in [1,2]:
                    n_0 = n_result['value'][(n_result['i']==i) & (n_result['t']==t-1) & (n_result['d']==d) & (n_result['j'] ==j)].tolist()[0]
                    y_ij_1 = y_result['value'][(y_result['i']==i) & (y_result['j'] ==j-1) & (y_result['i_d']==i) & (y_result['j_d'] ==j) & (y_result['t']==t-1) & (y_result['d']==d)].tolist()[0]
                    y_i_1j_1 = y_result['value'][(y_result['i']==i-1) & (y_result['j'] ==j-1) & (y_result['i_d']==i) & (y_result['j_d'] ==j) & (y_result['t']==t-1) & (y_result['d']==d)].tolist()[0]
                    y_i11j_1 = y_result['value'][(y_result['i']==i+1) & (y_result['j'] ==j-1) & (y_result['i_d']==i) & (y_result['j_d'] ==j) & (y_result['t']==t-1) & (y_result['d']==d)].tolist()[0]

                    y_ij11 = y_result['value'][(y_result['i']==i) & (y_result['j'] ==j) & (y_result['i_d']==i) & (y_result['j_d'] ==j+1) & (y_result['t']==t-1) & (y_result['d']==d)].tolist()[0]
                    y_i_1j11 = y_result['value'][(y_result['i']==i) & (y_result['j'] ==j) & (y_result['i_d']==i-1) & (y_result['j_d'] ==j+1) & (y_result['t']==t-1) & (y_result['d']==d)].tolist()[0]
                    y_i11j11 = y_result['value'][(y_result['i']==i) & (y_result['j'] ==j) & (y_result['i_d']==i+1) & (y_result['j_d'] ==j+1) & (y_result['t']==t-1) & (y_result['d']==d)].tolist()[0]

                    n_1 = n_0 + (y_ij_1 + y_i_1j_1 + y_i11j_1) - (y_ij11 + y_i_1j11 + y_i11j11)

                    n_result = n_result.append(pd.DataFrame({'i':[i],'j':[j],'d':[d],'t':[t],'value':[n_1]}),ignore_index=True)


            
        

        MODEL = gurobipy.Model("Lane_Manage")
        MODEL.Params.timeLimit = 300
        MODEL.Params.MIPGap = 0.1
        MODEL.params.NonConvex = 2
        MODEL.params.ConcurrentMIP = 4

        ##################################################

        #index

        T_end = 6
        J_end = 6
        I_end = 4
        I = range(I_end)
        J = range(J_end+2)
        T = range(T_end)
        D = [-1,1]#through left

        n = {}
        y = {}
        s = {}
        r = {}
        k = {}
        q = {}
        z = {}
        h = {}
        r_list = {}
        s_list = {}
        aux_r = {}
        aux_n = {}
        v_t = {}
        v_t_1 = {}

        k_max = 324.3 #veh/mile
        q_max = 3600 #veh/h
        v = 40 #40mile
        w = 15.365 
        delta_t = 5/3600 #per h
        delta_l =  v*delta_t # mile




        for i in I:
            for j in J:
                for d in D:
                    for t in T:
                        n[i,j,d,t] = MODEL.addVar(name='n_%s_%s_%s_%s' % (i,j,d,t))
                        k[i,j,d,t] = MODEL.addVar(name='k_%s_%s_%s_%s' % (i,j,d,t))
                        z[i,j,d,t] = MODEL.addVar(vtype=GRB.BINARY, name='z_%s_%s_%s_%s' % (i,j,d,t))
                        r_list[i,j,d,t] = MODEL.addVar(name='r_list_%s_%s_%s_%s' % (i,j,d,t))
                        s_list[i,j,d,t] = MODEL.addVar(name='s_list_%s_%s_%s_%s' % (i,j,d,t))
                        aux_r[i,j,d,t] = MODEL.addVar(name='aux_r_%s_%s_%s_%s' % (i,j,d,t))
                        aux_n[i,j,d,t] = MODEL.addVar(name='aux_n_%s_%s_%s_%s' % (i,j,d,t))
                        v_t[i,j,t] = MODEL.addVar(name='v_t_%s_%s_%s' % (i,j,t))
                        v_t_1[i,j,t] = MODEL.addVar(name='v_t_1_%s_%s_%s' % (i,j,t))
                        
                        for i_d in I:
                            j_d = j+1
                            if (i_d - i <= 1) and (i_d - i >= -1):
                                q[i,j,i_d,j_d,d,t] = MODEL.addVar(name='q_%s_%s_%s_%s_%s_%s' % (i,j,i_d,j_d,d,t))
                                y[i,j,i_d,j_d,d,t] = MODEL.addVar(name='y_%s_%s_%s_%s_%s_%s' % (i,j,i_d,j_d,d,t))
                                r[i,j,i_d,j_d,d,t] = MODEL.addVar(name='r_%s_%s_%s_%s_%s_%s' % (i,j,i_d,j_d,d,t))
                                s[i,j,i_d,j_d,d,t] = MODEL.addVar(name='s_%s_%s_%s_%s_%s_%s' % (i,j,i_d,j_d,d,t))
                                h[i,j,i_d,j_d,d,t] = MODEL.addVar(vtype=GRB.BINARY,name='h_%s_%s_%s_%s_%s_%s' % (i,j,i_d,j_d,d,t))


        MODEL.update()


        n_last_sum = quicksum((T_end-t)*n[i,J_end-1,d,t] for i in I for t in T for d in D)
        cost_lane_change = quicksum(y[i,j,i-1,j+1,d,t] for i in [1,2,3] for t in T for d in D for j in J)+quicksum(y[i,j,i+1,j+1,d,t] for i in [0,1,2] for t in T for d in D for j in J)
        cost_j = quicksum((- j)*n[i,j,d,t] for i in I for t in T for d in D for j in range(2,J_end))
        y_1 = quicksum(y[i,j,i,j+1,d,t] for i in I for t in T for d in D for j in [1]) + quicksum(y[i,j,i+1,j+1,d,t] for i in [0,1,2] for t in T for d in D for j in [1]) + quicksum(y[i,j,i-1,j+1,d,t] for i in [3,1,2] for t in T for d in D for j in [1])
        obj = w_s*n_last_sum - w_lc*cost_lane_change + w_i*y_1



        MODEL.setObjective(obj, GRB.MAXIMIZE)

        for i in I:
            for t in T:
                for d in D:
                    MODEL.addConstr(n[i,J_end+1,d,t]==0)


        # input
        for i in I:
            for j in J:
                for d in D:
                    t=0
                    MODEL.addConstr(n[i,j,d,t]==n_result['value'][(n_result['i']==i) & (n_result['t']==sepa) & (n_result['d']==d) & (n_result['j']==j)].tolist()[0])

            


        b = MODEL.addVar(vtype=GRB.BINARY, name="b")

        # Constants
        eps = 0.00001
        M = delta_t*q_max + eps 
        for t in range(T_end):
            i=0
            y1[sepa+t] = round(y1[sepa+t])

            MODEL.addConstr(y1[sepa+t] >= r[i,0,i,1,dl1[sepa+t],t] + eps - M * (1 - b), name="bigM_constr1")
            MODEL.addConstr(y1[sepa+t] <= r[i,0,i,1,dl1[sepa+t],t] + M * b, name="bigM_constr2")

            # Add indicator constraints
            MODEL.addConstr((b == 1) >> (y[i,0,i,1,dl1[sepa+t],t] == r[i,0,i,1,dl1[sepa+t],t]), name="indicator_constr1")
            MODEL.addConstr((b == 0) >> (y[i,0,i,1,dl1[sepa+t],t] == y1[sepa+t]), name="indicator_constr2")
            MODEL.addConstr((y[i,0,i,1,-dl1[sepa+t],t] == 0))

            i=1
            y2[sepa+t] = round(y2[sepa+t])

            MODEL.addConstr(y2[sepa+t] >= r[i,0,i,1,dl2[sepa+t],t] + eps - M * (1 - b), name="bigM_constr1")
            MODEL.addConstr(y2[sepa+t] <= r[i,0,i,1,dl2[sepa+t],t] + M * b, name="bigM_constr2")

            # Add indicator constraints
            MODEL.addConstr((b == 1) >> (y[i,0,i,1,dl2[sepa+t],t] == r[i,0,i,1,dl1[sepa+t],t]), name="indicator_constr1")
            MODEL.addConstr((b == 0) >> (y[i,0,i,1,dl2[sepa+t],t] == y2[sepa+t]), name="indicator_constr2")
            MODEL.addConstr((y[i,0,i,1,-dl2[sepa+t],t] == 0))

            i=2
            y3[sepa+t] = round(y3[sepa+t])

            MODEL.addConstr(y3[sepa+t] >= r[i,0,i,1,dl3[sepa+t],t] + eps - M * (1 - b), name="bigM_constr1")
            MODEL.addConstr(y3[sepa+t] <= r[i,0,i,1,dl3[sepa+t],t] + M * b, name="bigM_constr2")

            # Add indicator constraints
            MODEL.addConstr((b == 1) >> (y[i,0,i,1,dl3[sepa+t],t] == r[i,0,i,1,dl1[sepa+t],t]), name="indicator_constr1")
            MODEL.addConstr((b == 0) >> (y[i,0,i,1,dl3[sepa+t],t] == y3[sepa+t]), name="indicator_constr2")
            MODEL.addConstr((y[i,0,i,1,-dl3[sepa+t],t] == 0))

            i=3
            y4[sepa+t] = round(y4[sepa+t])

            MODEL.addConstr(y4[sepa+t] >= r[i,0,i,1,dl4[sepa+t],t] + eps - M * (1 - b), name="bigM_constr1")
            MODEL.addConstr(y4[sepa+t] <= r[i,0,i,1,dl4[sepa+t],t] + M * b, name="bigM_constr2")

            # Add indicator constraints
            MODEL.addConstr((b == 1) >> (y[i,0,i,1,dl4[sepa+t],t] == r[i,0,i,1,dl1[sepa+t],t]), name="indicator_constr1")
            MODEL.addConstr((b == 0) >> (y[i,0,i,1,dl4[sepa+t],t] == y4[sepa+t]), name="indicator_constr2")
            MODEL.addConstr((y[i,0,i,1,-dl4[sepa+t],t] == 0))

            

        for d in D:
            for i in I:
                for t in T:
                    MODEL.addConstr(n[i,0,d,t]==0)

        



        for i in I:
            for t in T:
                sg = ss[sepa][t]
                if sg == 1:
                    MODEL.addConstr(y[i,J_end-2,i,J_end-1,1,t]<=0)
                elif sg == 2:
                    MODEL.addConstr(y[i,J_end-2,i,J_end-1,-1,t]<=0)
                elif sg == 0:
                    MODEL.addConstr(y[i,J_end-2,i,J_end-1,1,t]<=0)
                    MODEL.addConstr(y[i,J_end-2,i,J_end-1,-1,t]<=0)

        #cannot cross lane change
        for j in J:
            for t in T:
                for d in D:
                    i=0
                    MODEL.addConstr(y[i,j,i+1,j+1,d,t] <= h[i,j,i+1,j+1,d,t]*y[i,j,i+1,j+1,d,t])
                    MODEL.addConstr(y[i,j,i,j+1,d,t] <= h[i,j,i,j+1,d,t]*y[i,j,i,j+1,d,t])
                    i=I_end-1
                    MODEL.addConstr(y[i,j,i-1,j+1,d,t] <= h[i,j,i-1,j+1,d,t]*y[i,j,i-1,j+1,d,t])
                    MODEL.addConstr(y[i,j,i,j+1,d,t] <= h[i,j,i,j+1,d,t]*y[i,j,i,j+1,d,t])
                    for i in range(1,I_end-1):
                        for i_d in [i+1,i-1,i]:
                            MODEL.addConstr(y[i,j,i_d,j+1,d,t] <= h[i,j,i_d,j+1,d,t]*y[i,j,i_d,j+1,d,t])


        for j in range(0,J_end+1):
            for t in T:
                i = 0
                MODEL.addConstr(quicksum(h[i,j,i+1,j+1,d,t]+h[i+1,j,i,j+1,d,t] for d in D) <= 1)
                i = I_end-1
                MODEL.addConstr(quicksum(h[i,j,i-1,j+1,d,t]+h[i-1,j,i,j+1,d,t] for d in D) <= 1)
                for i in range(1,I_end-1):
                    MODEL.addConstr(quicksum(h[i,j,i+1,j+1,d,t]+h[i+1,j,i,j+1,d,t] for d in D) <= 1)
                    MODEL.addConstr(quicksum(h[i,j,i-1,j+1,d,t]+h[i-1,j,i,j+1,d,t] for d in D) <= 1)

        for j in range(0,J_end+1):
            for t in T:
                for d in D:
                    i = 0
                    MODEL.addConstr(h[i,j,i,j+1,d,t]+h[i+1,j,i,j+1,-d,t] <= 1)
                    i = I_end-1
                    MODEL.addConstr(h[i,j,i,j+1,d,t]+h[i-1,j,i,j+1,-d,t] <= 1)
                    for i in range(1,I_end-1):
                        MODEL.addConstr(h[i,j,i,j+1,d,t]+h[i+1,j,i,j+1,-d,t]  <= 1)
                        MODEL.addConstr(h[i,j,i,j+1,d,t]+h[i-1,j,i,j+1,-d,t]  <= 1)
                        MODEL.addConstr(h[i-1,j,i,j+1,d,t]+h[i+1,j,i,j+1,-d,t]  <= 1)
                    

        for i in I:
            for j in J:
                for t in T:
                    for d in D:
                        MODEL.addConstr(n[i,j,d,t] <= z[i,j,d,t]*n[i,j,d,t])

        for i in I:
            for j in range(2,J_end+2):
                for t in T:
                    MODEL.addConstr(quicksum(z[i,j,d,t] for d in D) <= 1)

        for i in I:
            for j in [0,1]:
                for t in T:
                    MODEL.addConstr(quicksum(z[i,j,d,t] for d in D) <= 2)

        for t in T:
            for j in [0, J_end-1,J_end-2]:
                for d in D:
                    for i in range(1,I_end-1):
                        MODEL.addConstr(y[i,j,i+1,j+1,d,t]==0)
                        MODEL.addConstr(y[i,j,i-1,j+1,d,t]==0)
                        MODEL.addConstr(y[i,j,i,j+1,d,t]>=0)
                    i=0
                    MODEL.addConstr(y[i,j,i+1,j+1,d,t]==0)
                    MODEL.addConstr(y[i,j,i,j+1,d,t]>=0)
                    i=I_end-1
                    MODEL.addConstr(y[i,j,i-1,j+1,d,t]==0)
                    MODEL.addConstr(y[i,j,i,j+1,d,t]>=0)



        #same v in a cell
        for j in [0,1]:
            for t in T:
                i = 0
                MODEL.addConstr((y[i,j,i,j+1,1,t]+y[i,j,i+1,j+1,1,t])*n[i,j,-1,t] == (y[i,j,i,j+1,-1,t]+y[i,j,i+1,j+1,-1,t])*n[i,j,1,t])
                i = I_end-1
                MODEL.addConstr((y[i,j,i-1,j+1,1,t]+y[i,j,i,j+1,1,t])*n[i,j,-1,t] == (y[i,j,i-1,j+1,-1,t]+y[i,j,i,j+1,-1,t])*n[i,j,1,t])
                for i in range(1,I_end-1):
                    MODEL.addConstr((y[i,j,i-1,j+1,1,t]+y[i,j,i,j+1,1,t]+y[i,j,i+1,j+1,1,t])*n[i,j,-1,t] == (y[i,j,i-1,j+1,-1,t]+y[i,j,i,j+1,-1,t]+y[i,j,i+1,j+1,-1,t])*n[i,j,1,t])
                
                    


        #dynamic
        #############################
        #set receiving flow R, sending flow S
        for j in range(1,J_end+1):
            for t in T:
                for i in I:
                    MODEL.addConstr(aux_n[i,j,d,t] == n[i,j,1,t]+n[i,j,-1,t])
                    MODEL.addGenConstrMin(s_list[i,j,d,t],[aux_n[i,j,d,t],q_max*delta_t])

                    i = 0   
                    MODEL.addConstr(quicksum(s[i,j,i,j+1,d,t] + s[i,j,i+1,j+1,d,t] for d in D)== s_list[i,j,d,t])

                    i = 3    
                    MODEL.addConstr(quicksum(s[i,j,i,j+1,d,t] + s[i,j,i-1,j+1,d,t] for d in D)== s_list[i,j,d,t])

                    for i in [1,2]:   
                        MODEL.addConstr(quicksum(s[i,j,i,j+1,d,t] + s[i,j,i-1,j+1,d,t] + s[i,j,i+1,j+1,d,t] for d in D) == s_list[i,j,d,t])


        for j in range(J_end+1):
            for t in T:
                for i in I:
                    MODEL.addConstr(aux_r[i,j,d,t] == w/v*(k_max*delta_l-(n[i,j,1,t]+n[i,j,-1,t])))
                    MODEL.addGenConstrMin(r_list[i,j,d,t],[aux_r[i,j,d,t],q_max*delta_t])

                    i = 0
                    MODEL.addConstr(quicksum(r[i,j,i,j+1,d,t] + r[i+1,j,i,j+1,d,t] for d in D) == r_list[i,j+1,d,t])

                    i = 3
                    MODEL.addConstr(quicksum(r[i,j,i,j+1,d,t] + r[i-1,j,i,j+1,d,t] for d in D) ==r_list[i,j+1,d,t])

                    
                    for i in [1,2]: 
                        MODEL.addConstr(quicksum(r[i,j,i,j+1,d,t] + r[i-1,j,i,j+1,d,t] + r[i+1,j,i,j+1,d,t] for d in D)==r_list[i,j+1,d,t])

        for j in range(J_end+1):
            for t in T:
                for d in D:
                    for i in I:
                        for i_d in I:
                            if (i_d - i <= 1) and (i_d - i >= -1):
                                MODEL.addGenConstrMin(y[i,j,i_d,j+1,d,t],[s[i,j,i_d,j+1,d,t],r[i,j,i_d,j+1,d,t]])




        for j in range(1,J_end+1):
            for t in T:
                for d in D:
                    i=0
                    MODEL.addConstr(n[i,j,d,t] >= y[i,j,i,j+1,d,t]+y[i,j,i+1,j+1,d,t])
                    i=I_end-1
                    MODEL.addConstr(n[i,j,d,t] >= y[i,j,i,j+1,d,t]+y[i,j,i-1,j+1,d,t])
                    for i in range(1,I_end-1):
                        MODEL.addConstr(n[i,j,d,t] >= y[i,j,i,j+1,d,t]+y[i,j,i-1,j+1,d,t]+y[i,j,i+1,j+1,d,t]) 

        for j in range(1,J_end+1):
            for t in range(1,T_end):
                for d in D:
                    i=0
                    MODEL.addConstr(n[i,j,d,t] == n[i,j,d,t-1]+(y[i,j-1,i,j,d,t-1]+y[i+1,j-1,i,j,d,t-1])-(y[i,j,i,j+1,d,t-1]+y[i,j,i+1,j+1,d,t-1]))
                    i=I_end-1
                    MODEL.addConstr(n[i,j,d,t] == n[i,j,d,t-1]+(y[i,j-1,i,j,d,t-1]+y[i-1,j-1,i,j,d,t-1])-(y[i,j,i,j+1,d,t-1]+y[i,j,i-1,j+1,d,t-1]))

                    for i in range(1,I_end-1):
                        MODEL.addConstr(n[i,j,d,t] == n[i,j,d,t-1]+(y[i-1,j-1,i,j,d,t-1]+y[i,j-1,i,j,d,t-1]+y[i+1,j-1,i,j,d,t-1])-(y[i,j,i-1,j+1,d,t-1]+y[i,j,i,j+1,d,t-1]+y[i,j,i+1,j+1,d,t-1]))



        for i in I:
            for j in J:
                for t in T:
                    for d in D:
                        MODEL.addConstr(n[i,j,d,t] == k[i,j,d,t]*delta_l)
                        for i_d in I:
                            if (i_d - i <= 1) and (i_d - i >= -1):
                                MODEL.addConstr(y[i,j,i_d,j+1,d,t] == q[i,j,i_d,j+1,d,t]*delta_t)
                                MODEL.addConstr(q[i,j,i_d,j+1,d,t] >=0)
                                MODEL.addConstr(quicksum(q[i,j,i_d,j+1,d,t] for d in D) <=q_max)
        for i in I:
            for j in J:
                for t in T:
                    MODEL.addConstr(quicksum(k[i,j,d,t] for d in D) <= k_max)
                    for d in D:
                        MODEL.addConstr(k[i,j,d,t] >= 0)




        for i in I:
            for d in D:
                for t in range(1,T_end):
                        MODEL.addConstr(n[i,J_end-1,d,t-1] == n[i,J_end,d,t])
                        MODEL.addConstr(n[i,J_end-1,d,t-1] == y[i,J_end-1,i,J_end,d,t-1])

        for tt in T:
            MODEL.addConstr(quicksum(y[i,0,i,1,d,t]for t in range(tt) for i in I for d in D) +  quicksum(n[i,j,d,0] for i in I for j in range(J_end-1) for d in D) == quicksum(y[i,J_end-2,i,J_end-1,d,t]
            for t in range(tt) for i in I for d in D) + quicksum(n[i,j,d,tt] for i in I for j in range(J_end-1) for d in D))

        #################################
        #add a related constr, v=f/d, a=(v-v)/t
        for i in I:
            for j in range(1,J_end+1):
                for t in range(0,T_end-1):
                    eligible_i_d = [i_d for i_d in I if (i_d - i <= 1) and (i_d - i >= -1)]
                    if eligible_i_d:
                        numerator = quicksum(q[i,j,i_d,j+1,d,t] for i_d in eligible_i_d for d in D)
                        denominator = quicksum(k[i,j,d,t] for d in D)
                        MODEL.addConstr(numerator == v_t[i,j,t] * denominator)

                        numerator_1 = quicksum(q[i,j,i_d,j+1,d,t+1] for i_d in eligible_i_d for d in D)
                        denominator_1 = quicksum(k[i,j,d,t+1] for d in D)
                        MODEL.addConstr(numerator_1 == v_t_1[i,j,t+1] * denominator_1)

                        MODEL.addConstr(v_t_1[i,j,t+1] - v_t[i,j,t+1] <=  a_max*delta_t)
                        MODEL.addConstr(v_t_1[i,j,t+1] - v_t[i,j,t+1] >= - a_max*delta_t)




        


        MODEL.optimize()
        
        n_result = n_result.drop(n_result[n_result['t']>sepa].index)

        y_result = y_result.drop(y_result[y_result['t']>=sepa].index)



        
        for j in J:
            for i in I:
                for d in D:
                    for t in range(1,T_end):
                        value_n = n[i,j,d,t].X
                        if value_n < 0.000000001:
                            value_n =0
                            n_result = n_result.append(pd.DataFrame({'i':[i],'j':[j],'d':[d],'t':[t+sepa],'value':[value_n]}),ignore_index=True)
                        else:
                            n_result = n_result.append(pd.DataFrame({'i':[i],'j':[j],'d':[d],'t':[t+sepa],'value':[value_n]}),ignore_index=True)
                            
                        value_k = k[i,j,d,t].X
                        k_result = k_result.append(pd.DataFrame({'i':[i],'j':[j],'d':[d],'t':[t+sepa],'value':[value_k]}),ignore_index=True)

                    for t in T:
                        for i_d in I:
                            if (i_d - i <= 1) and (i_d - i >= -1):
                                value_y = round(y[i,j,i_d,j+1,d,t].X)
                                if value_y < 0.000000001:
                                    value_y =0
                                    y_result = y_result.append(pd.DataFrame({'i':[i],'j':[j],'i_d':[i_d],'j_d':[j+1],'d':[d],'t':[t+sepa],'value':[value_y]}),ignore_index=True)
                                else:
                                    y_result = y_result.append(pd.DataFrame({'i':[i],'j':[j],'i_d':[i_d],'j_d':[j+1],'d':[d],'t':[t+sepa],'value':[value_y]}),ignore_index=True)

                                value_q = q[i,j,i_d,j+1,d,t].X
                                q_result = q_result.append(pd.DataFrame({'i':[i],'j':[j],'i_d':[i_d],'j_d':[j+1],'d':[d],'t':[t+sepa],'value':[value_q]}),ignore_index=True)


    thr = through(n_result, sepa)
    return(n_result,y_result,sepa,J_end,I_end,thr)

